In [1]:
import json
import os
from datetime import datetime, timedelta
from collections import defaultdict
import re

In [ ]:
#version 1
class Transaction:
    
    def __init__(self, amount, description, category=None, date=None):
        self.id = self._generate_id()
        self.amount = float(amount)
        self.description = description
        self.category = category or "Other"
        self.date = date or datetime.now().strftime("%Y-%m-%d")
        self.timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    def _generate_id(self):
        return f"T{datetime.now().strftime('%Y%m%d%H%M%S')}"
    
    def to_dict(self):
        return {
            'id': self.id,
            'amount': self.amount,
            'description': self.description,
            'category': self.category,
            'date': self.date,
            'timestamp': self.timestamp
             }
            
class SmartCategoryClassifier:
    
    def __init__(self):
        self.category_keywords = {
            'Food&Drinks': [ 'Coffee', 'Milk Tea', 'Breakfast','Lunch','Dinner','Canteen', 'McDonalds', 'Starbucks', 'Uber Eats', 'deliveroo'],
            'Transportation': ['Subway', 'Bus', 'Taxi', 'Gas Cost', 'Parking', 'Train Ticket', 'Air Ticket'],
            'Shopping': ['Supermarket', 'Shoes', 'Clothing', 'Skin Care Products', 'Electronic Products', 'Amazon'],
            'Entertainment': ['Movies', 'Games', 'KTV', 'Travel', 'Gym', 'Bookstore', 'Concert'],
            'Medical': ['Hospital', 'Pharmacy', 'Physical Examination', 'Dentist', 'Medicine'],
            'Education': ['Tuition', 'Tutoring', 'Exam Fees', 'Textbooks'],
            'Life Expense': ['Rent', 'Utilities', 'Internet', 'Furniture'],
            'Other': []
        }
    
    def suggest_category(self, description):
        # Match Words Case-insensitively
        description_lower = description.lower()
        
        # Match Keywords
        scores = {}
        for category, keywords in self.category_keywords.items():
            score = 0
            for keyword in keywords:
                if keyword.lower() in description_lower:
                    score += 2
            scores[category] = score
        
        if max(scores.values()) > 0:
            return max(scores, key=scores.get)
        return 'Other'

class SpendingAnalyzer:
    """a spending analyzer"""
    def analyze_monthly_spending(self, transactions):
        if not transactions:
            return {"error": "no data"}
        
        # statistics by category
        category_totals = defaultdict(float)
        total_spending = 0
        
        for trans in transactions:
            category_totals[trans.category] += trans.amount
            total_spending += trans.amount
        
        # calculate the proportion of spendings
        category_percentages = {}
        for category, amount in category_totals.items():
            percentage = (amount / total_spending) * 100 if total_spending > 0 else 0
            category_percentages[category] = {
                'amount': amount,
                'percentage': round(percentage, 1)
            }
        
        return {
            'total_spending': total_spending,
            'category_breakdown': category_percentages,
            'transaction_count': len(transactions),
            'average_transaction': round(total_spending / len(transactions), 2) if transactions else 0
        }
    
    
    def generate_spending_advice(self, analysis):
        """generate spending advice"""
        if 'error' in analysis:
            return ["no data for advices"]
        
        advice = []
        category_breakdown = analysis['category_breakdown']
        
        for category, data in category_breakdown.items():
            percentage = data['percentage']
            
            if category == 'Food&Drinks' and percentage > 35:
                advice.append(f"🍽️ The proportion of dining expenses is relatively high at {percentage}%")
            elif category == 'Entertainment' and percentage > 25:
                advice.append(f"🎮 The proportion of entertainment expenditure is relatively high at {percentage}%")
            elif category == 'Shopping' and percentage > 30:
                advice.append(f"🛍️ The proportion of shopping expenditure, {percentage}%, is on the high side")
            elif category == 'Transportation' and percentage > 20:
                advice.append(f"🚌 The proportion of transportation expenditure, {percentage}%, is on the high side")
        
        if analysis['average_transaction'] > 100:
            advice.append(f"The average single transaction cost of {analysis ['average_transaction']} is relatively high, and it is recommended to control consumption.")
        
        if not advice:
            advice.append(f"Your consumption structure is quite reasonable, continue to maintain it!")
        
        return advice
            
class BudgetSystem:
    """Simplified Budget System"""
    
    def __init__(self):
        self.transactions = []  # Simplified: store in memory only
        self.classifier = SmartCategoryClassifier()
        self.analyzer = SpendingAnalyzer()
    
    def add_expense(self):
        """1. Add Expense Record"""
        print("\n💰 Add Expense Record")
        print("-" * 30)
        
        try:
            amount = float(input("Enter amount: $"))
            if amount <= 0:
                print("❌ Amount must be greater than 0!")
                return
            
            description = input("Enter expense description: ").strip()
            if not description:
                print("❌ Description cannot be empty!")
                return
            
            # Smart category suggestion
            suggested_category = self.classifier.suggest_category(description)
            print(f"\n🤖 Smart category suggestion: {suggested_category}")
            
            categories = ['Food&Drinks', 'Transportation', 'Shopping', 'Entertainment', 'Medical', 'Education', 'Housing', 'Other']
            print("Available categories:", " | ".join(categories))
            user_category = input(f"Confirm category (press Enter to use '{suggested_category}'): ").strip()
            
            final_category = user_category if user_category in categories else suggested_category
            
            # Create and save transaction record
            transaction = Transaction(amount, description, final_category)
            self.transactions.append(transaction)
            
            print(f"✅ Successfully added expense record:")
            print(f"   Amount: ${amount}")
            print(f"   Description: {description}")
            print(f"   Category: {final_category}")
            print(f"   Time: {transaction.timestamp}")
            
        except ValueError:
            print("❌ Please enter a valid amount!")
        except Exception as e:
            print(f"❌ Failed to add record: {e}")
    
    def view_transactions(self):
        """2. View Transaction Records"""
        print("\n📊 View Transaction Records")
        print("-" * 50)
        
        if not self.transactions:
            print("No transaction records")
            return
        
        print(f"Total {len(self.transactions)} records:\n")
        
        # Display in reverse chronological order
        sorted_transactions = sorted(self.transactions, key=lambda x: x.timestamp, reverse=True)
        
        for i, trans in enumerate(sorted_transactions, 1):
            print(f"{i}. ${trans.amount} | {trans.description} | {trans.category} | {trans.timestamp}")
        
        total = sum(trans.amount for trans in self.transactions)
        print(f"\n💸 Total Spending: ${total:.2f}")
    
    def analyze_spending(self):
        """3. Spending Analysis Report"""
        print("\n📈 Spending Analysis Report")
        print("=" * 50)
        
        if not self.transactions:
            print("No data available for analysis")
            return
        
        analysis = self.analyzer.analyze_monthly_spending(self.transactions)
        
        if 'error' in analysis:
            print("Analysis failed:", analysis['error'])
            return
        
        # Basic statistics
        print(f"📊 Basic Statistics:")
        print(f"   Total Spending: ${analysis['total_spending']:.2f}")
        print(f"   Transaction Count: {analysis['transaction_count']} times")
        print(f"   Average per Transaction: ${analysis['average_transaction']:.2f}")
        print()
        
        # Category breakdown
        print("📋 Category Breakdown:")
        category_breakdown = analysis['category_breakdown']
        sorted_categories = sorted(category_breakdown.items(), 
                                 key=lambda x: x[1]['amount'], reverse=True)
        
        for category, data in sorted_categories:
            amount = data['amount']
            percentage = data['percentage']
            bar_length = int(percentage / 5)  # One character per 5%
            bar = "█" * bar_length
            print(f"   {category}: ${amount:>8.2f} ({percentage:>5.1f}%) {bar}")
        
        print()
        
        # Smart advice
        advice = self.analyzer.generate_spending_advice(analysis)
        print("💡 Spending Advice:")
        for i, suggestion in enumerate(advice, 1):
            print(f"   {i}. {suggestion}")
    
    def show_menu(self):
        """Display Main Menu"""
        print("\n" + "=" * 50)
        print("Smart Budget System - Main Menu")
        print("=" * 50)
        print("1. 💰 Add Expense Record")
        print("2. 📊 View Transaction Records")
        print("3. 📈 Spending Analysis Report")
        print("0. 🚪 Exit System")
        print("-" * 50)
    
    def run(self):
        """Run Main Program"""
        print("Starting Smart Budget System...")
        print("Smart Budget System is ready!")
        print("Welcome to Smart Budget System!")
        
        while True:
            self.show_menu()
            
            try:
                choice = input("Please select function (0-3): ").strip()
                
                if choice == '1':
                    self.add_expense()
                elif choice == '2':
                    self.view_transactions()
                elif choice == '3':
                    self.analyze_spending()
                elif choice == '0':
                    print("\n👋 Thank you for using Smart Budget System, goodbye!")
                    break
                else:
                    print("❌ Invalid selection, please enter 0-3")
                
                input("\nPress Enter to continue...")
                
            except KeyboardInterrupt:
                print("\n\n👋 Program exited")
                break
            except Exception as e:
                print(f"❌ An error occurred: {e}")

def main():
    """Main Program Entry"""
    budget_system = BudgetSystem()
    budget_system.run()

if __name__ == "__main__":
    main()

Starting Smart Budget System...
Smart Budget System is ready!
Welcome to Smart Budget System!

Smart Budget System - Main Menu
1. 💰 Add Expense Record
2. 📊 View Transaction Records
3. 📈 Spending Analysis Report
0. 🚪 Exit System
--------------------------------------------------
Please select function (0-3): 1

💰 Add Expense Record
------------------------------
Enter amount: $20
Enter expense description: Bought a cup of coffee 

🤖 Smart category suggestion: Food&Drinks
Available categories: Food&Drinks | Transportation | Shopping | Entertainment | Medical | Education | Housing | Other
Confirm category (press Enter to use 'Food&Drinks'): 
✅ Successfully added expense record:
   Amount: $20.0
   Description: Bought a cup of coffee
   Category: Food&Drinks
   Time: 2025-10-04 13:40:17

Press Enter to continue...

Smart Budget System - Main Menu
1. 💰 Add Expense Record
2. 📊 View Transaction Records
3. 📈 Spending Analysis Report
0. 🚪 Exit System
--------------------------------------------

In [ ]:
#version 2 Smart Budget System with Date Input Support


from datetime import datetime, timedelta
from collections import defaultdict

class Transaction:
    
    def __init__(self, amount, description, category=None, date=None):
        self.id = self._generate_id()
        self.amount = float(amount)
        self.description = description
        self.category = category or "Other"
        self.date = date or datetime.now().strftime("%Y-%m-%d")
        self.timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    def _generate_id(self):
        return f"T{datetime.now().strftime('%Y%m%d%H%M%S')}"
    
    def to_dict(self):
        return {
            'id': self.id,
            'amount': self.amount,
            'description': self.description,
            'category': self.category,
            'date': self.date,
            'timestamp': self.timestamp
        }

class DateHelper:
    """help deal with datetime"""
    
    @staticmethod
    def parse_date_from_description(description):
        description_lower = description.lower()
        today = datetime.now()
        

        if 'yesterday' in description_lower :
            return (today - timedelta(days=1)).strftime("%Y-%m-%d")
        elif 'today' in description_lower:
            return today.strftime("%Y-%m-%d")
        elif 'tomorrow' in description_lower:
            return (today + timedelta(days=1)).strftime("%Y-%m-%d")
        

        import re
        days_ago_pattern = re.search(r'(\d+)\s*days?\s*ago', description_lower)
        if days_ago_pattern:
            days = int(days_ago_pattern.group(1) or days_ago_pattern.group(2))
            return (today - timedelta(days=days)).strftime("%Y-%m-%d")
        
        return None
    
    @staticmethod
    def parse_date_input(date_input):
        if not date_input:
            return datetime.now().strftime("%Y-%m-%d")
        
        date_input = date_input.lower().strip()
        today = datetime.now()
        
        if date_input in ['today']:
            return today.strftime("%Y-%m-%d")
        elif date_input in ['yesterday']:
            return (today - timedelta(days=1)).strftime("%Y-%m-%d")
        elif date_input in ['tomorrow']:
            return (today + timedelta(days=1)).strftime("%Y-%m-%d")
        
        try:
            import re
            days_ago_match = re.match(r'(\d+)\s*days?\s*ago', date_input)
            if days_ago_match:
                days = int(days_ago_match.group(1))
                return (today - timedelta(days=days)).strftime("%Y-%m-%d")
        except ValueError:
            pass
        
        try:
            if len(date_input) == 10 and '-' in date_input:
                datetime.strptime(date_input, "%Y-%m-%d")
                return date_input
            elif len(date_input) == 5 and '-' in date_input:
                current_year = today.year
                full_date = f"{current_year}-{date_input}"
                datetime.strptime(full_date, "%Y-%m-%d")
                return full_date
            elif '/' in date_input:
                parts = date_input.split('/')
                if len(parts) == 2:
                    month, day = parts
                    current_year = today.year
                    full_date = f"{current_year}-{month.zfill(2)}-{day.zfill(2)}"
                    datetime.strptime(full_date, "%Y-%m-%d")
                    return full_date
        except ValueError:
            pass
        

        print(f"⚠️  Cannot parse date '{date_input}', using today's date")
        return today.strftime("%Y-%m-%d")
    
    @staticmethod
    def get_date_examples():
        return [
            "today",
            "yesterday", 
            "3 days ago",
            "2024-01-15",
            "1/15"
        ]


class SmartCategoryClassifier:
    
    def __init__(self):
        self.category_keywords = {
            'Food&Drinks': [ 'Coffee', 'Milk Tea', 'Breakfast','Lunch','Dinner','Canteen', 'McDonalds', 'Starbucks', 'Uber Eats', 'deliveroo'],
            'Transportation': ['Subway', 'Bus', 'Taxi', 'Gas Cost', 'Parking', 'Train Ticket', 'Air Ticket'],
            'Shopping': ['Supermarket', 'Shoes', 'Clothing', 'Skin Care Products', 'Electronic Products', 'Amazon'],
            'Entertainment': ['Movies', 'Games', 'KTV', 'Travel', 'Gym', 'Bookstore', 'Concert'],
            'Medical': ['Hospital', 'Pharmacy', 'Physical Examination', 'Dentist', 'Medicine'],
            'Education': ['Tuition', 'Tutoring', 'Exam Fees', 'Textbooks'],
            'Life Expense': ['Rent', 'Utilities', 'Internet', 'Furniture'],
            'Other': []
        }
    
    def suggest_category(self, description):
        # Match Words Case-insensitively
        description_lower = description.lower()
        
        # Match Keywords
        scores = {}
        for category, keywords in self.category_keywords.items():
            score = 0
            for keyword in keywords:
                if keyword.lower() in description_lower:
                    score += 2
            scores[category] = score
        
        if max(scores.values()) > 0:
            return max(scores, key=scores.get)
        return 'Other'

class SpendingAnalyzer:
    """a spending analyzer"""
    def analyze_monthly_spending(self, transactions):
        if not transactions:
            return {"error": "no data"}
        
        # statistics by category
        category_totals = defaultdict(float)
        total_spending = 0
        
        for trans in transactions:
            category_totals[trans.category] += trans.amount
            total_spending += trans.amount
        
        # calculate the proportion of spendings
        category_percentages = {}
        for category, amount in category_totals.items():
            percentage = (amount / total_spending) * 100 if total_spending > 0 else 0
            category_percentages[category] = {
                'amount': amount,
                'percentage': round(percentage, 1)
            }
        
        return {
            'total_spending': total_spending,
            'category_breakdown': category_percentages,
            'transaction_count': len(transactions),
            'average_transaction': round(total_spending / len(transactions), 2) if transactions else 0
        }
    
    
    def generate_spending_advice(self, analysis):
        """generate spending advice"""
        if 'error' in analysis:
            return ["no data for advices"]
        
        advice = []
        category_breakdown = analysis['category_breakdown']
        
        for category, data in category_breakdown.items():
            percentage = data['percentage']
            
            if category == 'Food&Drinks' and percentage > 35:
                advice.append(f"🍽️ The proportion of dining expenses is relatively high at {percentage}%")
            elif category == 'Entertainment' and percentage > 25:
                advice.append(f"🎮 The proportion of entertainment expenditure is relatively high at {percentage}%")
            elif category == 'Shopping' and percentage > 30:
                advice.append(f"🛍️ The proportion of shopping expenditure, {percentage}%, is on the high side")
            elif category == 'Transportation' and percentage > 20:
                advice.append(f"🚌 The proportion of transportation expenditure, {percentage}%, is on the high side")
        
        if analysis['average_transaction'] > 100:
            advice.append(f"The average single transaction cost of {analysis ['average_transaction']} is relatively high, and it is recommended to control consumption.")
        
        if not advice:
            advice.append(f"Your consumption structure is quite reasonable, continue to maintain it!")
        
        return advice
            
class BudgetSystem:
    """Enhanced Budget System with Date Support"""
    
    def __init__(self):
        self.transactions = []  # Simplified: store in memory only
        self.classifier = SmartCategoryClassifier()
        self.analyzer = SpendingAnalyzer()
        self.date_helper = DateHelper()  
    
    def add_expense(self):
        """1. Add Expense Record with Date Support"""
        print("\n💰 Add Expense Record")
        print("-" * 30)
        
        try:
            amount = float(input("Enter amount: $"))
            if amount <= 0:
                print("❌ Amount must be greater than 0!")
                return
            
            description = input("Enter expense description: ").strip()
            if not description:
                print("❌ Description cannot be empty!")
                return
            
            auto_detected_date = self.date_helper.parse_date_from_description(description)
            
            print(f"\n📅 Date Input")
            if auto_detected_date:
                print(f"🤖 Auto-detected date from description: {auto_detected_date}")
                use_auto_date = input(f"Use auto-detected date '{auto_detected_date}'? (y/n, press Enter for yes): ").strip().lower()
                if use_auto_date in ['', 'y', 'yes']:
                    expense_date = auto_detected_date
                else:
                    print("Date input examples:", " | ".join(self.date_helper.get_date_examples()))
                    date_input = input("Enter date manually: ").strip()
                    expense_date = self.date_helper.parse_date_input(date_input)
            else:
                print("Date input examples:", " | ".join(self.date_helper.get_date_examples()))
                date_input = input("Enter date (press Enter for today): ").strip()
                expense_date = self.date_helper.parse_date_input(date_input)
            
            print(f"📅 Date set to: {expense_date}")
            
            # Smart category suggestion
            suggested_category = self.classifier.suggest_category(description)
            print(f"\n🤖 Smart category suggestion: {suggested_category}")
            
            categories = ['Food&Drinks', 'Transportation', 'Shopping', 'Entertainment', 'Medical', 'Education', 'Life Expense', 'Other']
            print("Available categories:", " | ".join(categories))
            user_category = input(f"Confirm category (press Enter to use '{suggested_category}'): ").strip()
            
            final_category = user_category if user_category in categories else suggested_category
            
            # Create and save transaction record with date
            transaction = Transaction(amount, description, final_category, expense_date)
            self.transactions.append(transaction)
            
            print(f"✅ Successfully added expense record:")
            print(f"   Amount: ${amount}")
            print(f"   Description: {description}")
            print(f"   Category: {final_category}")
            print(f"   Date: {expense_date}")
            print(f"   Added at: {transaction.timestamp}")
            
        except ValueError:
            print("❌ Please enter a valid amount!")
        except Exception as e:
            print(f"❌ Failed to add record: {e}")
    
    def view_transactions(self):
        """2. View Transaction Records with Date Display"""
        print("\n📊 View Transaction Records")
        print("-" * 70)
        
        if not self.transactions:
            print("No transaction records")
            return
        
        print(f"Total {len(self.transactions)} records:\n")
        
        # Display in reverse chronological order (by date first, then timestamp)
        sorted_transactions = sorted(self.transactions, key=lambda x: (x.date, x.timestamp), reverse=True)
        
        print("No.  Amount    | Description          | Category        | Date       | Added Time")
        print("-" * 80)
        
        for i, trans in enumerate(sorted_transactions, 1):
            print(f"{i:<4} ${trans.amount:<8.2f} | {trans.description:<20} | {trans.category:<15} | {trans.date} | {trans.timestamp}")
        
        total = sum(trans.amount for trans in self.transactions)
        print(f"\n💸 Total Spending: ${total:.2f}")
    
    def analyze_spending(self):
        """3. Spending Analysis Report"""
        print("\n📈 Spending Analysis Report")
        print("=" * 50)
        
        if not self.transactions:
            print("No data available for analysis")
            return
        
        analysis = self.analyzer.analyze_monthly_spending(self.transactions)
        
        if 'error' in analysis:
            print("Analysis failed:", analysis['error'])
            return
        
        # Basic statistics
        print(f"📊 Basic Statistics:")
        print(f"   Total Spending: ${analysis['total_spending']:.2f}")
        print(f"   Transaction Count: {analysis['transaction_count']} times")
        print(f"   Average per Transaction: ${analysis['average_transaction']:.2f}")
        print()
        
        # Category breakdown
        print("📋 Category Breakdown:")
        category_breakdown = analysis['category_breakdown']
        sorted_categories = sorted(category_breakdown.items(), 
                                 key=lambda x: x[1]['amount'], reverse=True)
        
        for category, data in sorted_categories:
            amount = data['amount']
            percentage = data['percentage']
            bar_length = int(percentage / 5)  # One character per 5%
            bar = "█" * bar_length
            print(f"   {category}: ${amount:>8.2f} ({percentage:>5.1f}%) {bar}")
        
        print()
        
        # Smart advice
        advice = self.analyzer.generate_spending_advice(analysis)
        print("💡 Spending Advice:")
        for i, suggestion in enumerate(advice, 1):
            print(f"   {i}. {suggestion}")
        
        # time range analysis
        if self.transactions:
            dates = [trans.date for trans in self.transactions]
            earliest_date = min(dates)
            latest_date = max(dates)
            print(f"\n📅 Date Range: {earliest_date} to {latest_date}")
    
    def show_menu(self):
        """Display Main Menu"""
        print("\n" + "=" * 50)
        print("🧠 Smart Budget System - Main Menu")
        print("=" * 50)
        print("1. 💰 Add Expense Record")
        print("2. 📊 View Transaction Records") 
        print("3. 📈 Spending Analysis Report")
        print("0. 🚪 Exit System")
        print("-" * 50)
    
    def run(self):
        """Run Main Program"""
        print("Starting Smart Budget System...")
        print("Enhanced with Date Support!")
        print("Welcome to Smart Budget System!")
        print("Now you can input dates like 'coffee yesterday'!")
        
        while True:
            self.show_menu()
            
            try:
                choice = input("Please select function (0-3): ").strip()
                
                if choice == '1':
                    self.add_expense()
                elif choice == '2':
                    self.view_transactions()
                elif choice == '3':
                    self.analyze_spending()
                elif choice == '0':
                    print("\n👋 Thank you for using Smart Budget System, goodbye!")
                    break
                else:
                    print("❌ Invalid selection, please enter 0-3")
                
                input("\nPress Enter to continue...")
                
            except KeyboardInterrupt:
                print("\n\n👋 Program exited")
                break
            except Exception as e:
                print(f"❌ An error occurred: {e}")

def main():
    """Main Program Entry"""
    budget_system = BudgetSystem()
    budget_system.run()

if __name__ == "__main__":
    main()

Starting Smart Budget System...
Enhanced with Date Support!
Welcome to Smart Budget System!
Now you can input dates like 'coffee yesterday'!

🧠 Smart Budget System - Main Menu
1. 💰 Add Expense Record
2. 📊 View Transaction Records
3. 📈 Spending Analysis Report
0. 🚪 Exit System
--------------------------------------------------
Please select function (0-3): 1

💰 Add Expense Record
------------------------------
Enter amount: $20
Enter expense description: coffee yesterday 

📅 Date Input
🤖 Auto-detected date from description: 2025-10-11
Use auto-detected date '2025-10-11'? (y/n, press Enter for yes): 
📅 Date set to: 2025-10-11

🤖 Smart category suggestion: Food&Drinks
Available categories: Food&Drinks | Transportation | Shopping | Entertainment | Medical | Education | Life Expense | Other
Confirm category (press Enter to use 'Food&Drinks'): 
✅ Successfully added expense record:
   Amount: $20.0
   Description: coffee yesterday
   Category: Food&Drinks
   Date: 2025-10-11
   Added at: 2025